In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('Churn_Modelling.csv')

In [5]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"Num GPUs Available: {len(gpus)}")
    print(f"GPUs: {gpus}")
else:
    print("No GPU detected")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
if gpus:
    try:
        # Set the first GPU as visible (if more than one is available)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [ ]:
df["Geography"].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [ ]:
df["Gender"].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

l1 = LabelEncoder()

df["Gender"] = l1.fit_transform(df["Gender"])

In [ ]:
# Saving LabelEncoder object

import pickle

with open('label_encoder.pkl','wb') as file:
    pickle.dump(l1,file)

In [ ]:
print(l1.transform('Male'))

In [ ]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df["Exited"].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [ ]:
from sklearn.preprocessing import OneHotEncoder

o1 = OneHotEncoder()

dfg = o1.fit_transform(df[["Geography"]])

o1.get_feature_names_out(['Geography'])



array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [ ]:
with open('onehot_encoder.pkl','wb') as file:
    pickle.dump(o1,file)

In [ ]:
geo_encoded = pd.DataFrame(dfg.toarray(), columns=o1.get_feature_names_out(['Geography']))

geo_encoded.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [ ]:
data = df

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data = pd.concat([data.drop(columns=["Geography"],axis=1), geo_encoded], axis=1)

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,2,15647311,Hill,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,3,15619304,Onio,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,4,15701354,Boni,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,5,15737888,Mitchell,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
data.drop(columns=["RowNumber","CustomerId","Surname"],axis=1,inplace=True)

In [ ]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
X = data.drop(columns=["Exited"],axis=1)
y = data["Exited"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Remove Correlated Features

correlated_features = set()

correlation_matrix = data.corr()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

correlated_features

set()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
with open('standard_scaler.pkl','wb') as file:
    pickle.dump(sc,file)

In [ ]:
X_train.shape
X_test.shape

(2000, 12)

In [ ]:
# Traditional Machine Learning Model


# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier()

# model.fit(X_train, y_train)


# y_pred = model.predict(X_test)

# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# print(f"Accuracy : {accuracy_score(y_test, y_pred)*100}%")
# print(f"Confusion Matrix : \n{confusion_matrix(y_test, y_pred)}")
# print(f"Classification Report : \n{classification_report(y_test, y_pred)}")

In [ ]:
(X_train.shape[1],)

(12,)

In [ ]:
# Deep Learning Model

# Flow

# Sequential N/W -> Dense -> Activation Function -> Optimizer -> Loss Function -> Metrics

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Define Squential Model with Dense Layers and Activation Function with its Hidden Layers

model = Sequential(
    [
        Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

2024-07-13 04:52:54.217078: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-07-13 04:52:54.217121: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-07-13 04:52:54.217132: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-07-13 04:52:54.217708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-13 04:52:54.218096: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 961 (3.75 KB)

 Trainable params: 961 (3.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

from tensorflow.keras.optimizers import Adam

# Define Optimizer, Loss Function and Metrics

adam = Adam(learning_rate=0.01)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

# Define Callbacks for Tensorboard and Early Stopping

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((8000, 12), (8000,), (2000, 12), (2000,))

In [ ]:
# Train the Model

history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=100, 
    # batch_size=32, 
    callbacks=[tensorboard, early_stopping]
)

Epoch 1/100


2024-07-13 04:52:55.615523: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7562 - loss: 0.5033 - val_accuracy: 0.8430 - val_loss: 0.3668
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8579 - loss: 0.3547 - val_accuracy: 0.8445 - val_loss: 0.3683
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8517 - loss: 0.3536 - val_accuracy: 0.8610 - val_loss: 0.3492
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8593 - loss: 0.3423 - val_accuracy: 0.8595 - val_loss: 0.3550
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8622 - loss: 0.3382 - val_accuracy: 0.8610 - val_loss: 0.3447
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8578 - loss: 0.3417 - val_accuracy: 0.8615 - val_loss: 0.3494
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8658 - loss: 0.3332 - val_accuracy: 0.8595 - val_loss: 0.3457
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8633 - loss: 0.3316 - val_accuracy: 0.860

In [ ]:
# Save the Model

model.save('churn_model.h5')

In [ ]:
def train_model(neurons, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(layers):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=train_model, epochs=50, batch_size=10, verbose=0)

from sklearn.model_selection import GridSearchCV

param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]

}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

In [ ]:
# Load the Model

%load_ext tensorboard

%tensorboard --logdir logs/fit
